In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import re
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('../Dataset/dataset1.csv')

In [3]:
# Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would", "shall", "could", "might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words = set(stop_words)
print(stop_words)


{'hadn', 'can', 'are', 'theirs', "you're", 'themselves', 'these', 'was', 'the', 'once', 'been', 'ours', 'herself', 'because', 'on', 'them', 'only', 'an', "hasn't", 'they', 'might', "you've", 'could', 'yourself', 'than', 'wouldn', "she's", 'here', 'itself', "isn't", 'each', 'through', 'when', 'd', 'why', 'wasn', 'll', 'further', 'such', 'does', "that'll", 'before', "mustn't", 'me', 'having', 'between', 'who', 'from', 'shan', 'to', 'if', 'and', 'again', "it's", 'by', 'most', 'haven', 'over', 'do', 'all', 'mightn', 'were', 't', 'that', 'now', 'won', "aren't", 'doing', 'isn', 'with', 'no', 'too', 'my', 'have', 'its', 'had', "shouldn't", 'o', 'we', 'until', 'under', 'nor', 'at', 'below', 'she', 'where', "weren't", 'this', 'more', 'of', 'yours', "haven't", 'her', 'hers', 'be', 'above', "shan't", 'for', 'it', 'own', 'how', 'shouldn', 'about', 'didn', 'up', 'is', 'in', 'y', 'myself', 'their', 'weren', 'while', "mightn't", 'any', 'being', 'ourselves', 'aren', 'am', 'him', "couldn't", 'as', 'i',

In [4]:

# Removing special character
import re
def remove_special_character(content):
    return re.sub('\W+', ' ', content)  # or re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's


def remove_url(content):
    return re.sub(r'http\S+', '', content)

# Removing the stopwords from text


def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)


# Expansion of english contractions
# substitution of different stopwords from the reviews column with expanded words
# Create a dictionary to map contractions to their expanded forms
contraction_mapping = {
    r"won\'t": "would not",
    r"can\'t": "can not",
    r"don\'t": "do not",
    r"shouldn\'t": "should not",
    r"needn\'t": "need not",
    r"hasn\'t": "has not",
    r"haven\'t": "have not",
    r"weren\'t": "were not",
    r"mightn\'t": "might not",
    r"didn\'t": "did not",
    r"n\'t": " not",
    r"\'re": " are",
    r"\'s": " is",
    r"\'m": "am",
    r"\'ve": " have"
}



def contraction_expansion(content):
    
    for pattern, replacement in contraction_mapping.items():
        content = re.sub(pattern, replacement, content)

    return content

# Data preprocessing


def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)

    content = remove_stopwords(content)
    return content


In [5]:
# pd.options.display.max_colwidth = 1000
# Data cleaning
df['Reviews_clean'] = df['Reviews'].apply(data_cleaning)
df.head(1)


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,*Disclaimer: I only watched this movie as a co...,Disaster Movie,* Isenção de responsabilidade: eu só assisti e...,disclaimer watched movie conditional agreement...


In [6]:
# Mapping rating data to Binary label 1 (+ve) if rating >=7 and 0 (-ve) if rating <=4 and 2 (neutral) if rating = 5 or 6
df['Label'] = df['Ratings'].apply(
    lambda x: '1' if x >= 7 else ('0' if x <= 4 else '2'))
# After executing this line of code, the DataFrame df will contain only those rows where the 'Label' column satisfies the condition, and all other rows will be removed from the DataFrame.
df = df[df.Label < '2']
data = df[['Reviews_clean', 'Reviews', 'Ratings', 'Label']]
print(data['Label'].value_counts())


0    60000
1    60000
Name: Label, dtype: int64


In [7]:
# Importing dependencies for feature engineering
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer


In [8]:
# lemmatization of word
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()

    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]


In [9]:
train, test = train_test_split(
data, test_size=.3, random_state=42, shuffle=True)
# countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
tfidfvect = TfidfVectorizer(analyzer="word", tokenizer=LemmaTokenizer(
), ngram_range=(1, 3), min_df=10, max_features=10000)
# x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
# x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()

y_train = train['Label']
y_test = test['Label']


In [10]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5000
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram", "Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names_out())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number, category))
    print("\t# Unigrams :\n\t. %s" % ('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" % ('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" % ('\n\t. '.join(trigrams[-N:])))
    Number += 1


1. 1 :
	# Unigrams :
	. mental
	. vast
	. election
	. levy
	. vulgar
	. considerable
	. hyped
	. hears
	. minimal
	. literal
	. slave
	. pole
	. francis
	. jungle
	. spoken
	. gabriel
	. wrestling
	. damaged
	. sold
	. settled
	. correct
	. deadly
	. hannah
	. bounty
	. dennis
	. conclusion
	. marilyn
	. jam
	. forgetting
	. scored
	. dutch
	. mentioned
	. contract
	. realize
	. remade
	. educational
	. bent
	. contender
	. pop
	. fighting
	. motif
	. cliff
	. revolves
	. deputy
	. ape
	. caring
	. ritual
	. commit
	. splatter
	. chop
	. annie
	. profile
	. rope
	. onto
	. animal
	. giggle
	. missile
	. expression
	. coincidence
	. disease
	. explored
	. excited
	. transformed
	. yr
	. baddie
	. pray
	. recall
	. wizard
	. brando
	. hunter
	. say
	. posted
	. trump
	. daddy
	. egg
	. add
	. southern
	. climbing
	. lisa
	. resulting
	. casting
	. field
	. changing
	. praise
	. brazil
	. viking
	. ann
	. chainsaw
	. host
	. clue
	. holy
	. threatened
	. catholic
	. saint
	. sibling
	. aw

In [11]:
import sys
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline


In [12]:
model_1=LogisticRegression()

In [13]:
model_1.fit(x_train_tfidf,y_train)

y_pred1=model_1.predict(x_train_tfidf)

In [ ]:
print("Logistic Regression on Train: ")
print("Precision Score : %s" %
      precision_score(y_train, y_pred1, average='micro'))
print("AUC Score : %s" % roc_auc_score(
    y_train, model_1.predict_proba(x_train_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_train_1 = f1_score(y_train, y_pred1, average="weighted")
print("F1 Score : %s" % f1_score_train_1)
print("Logistic Regression on Test: ")
print("Precision Score : %s" %
      precision_score(y_test, y_pred1, average='micro'))
print("AUC Score : %s" % roc_auc_score(y_test,
      model_1.predict_proba(x_test_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_1 = f1_score(y_test, y_pred1, average="weighted")
print("F1 Score : %s" % f1_score_1)


Precision Score on training dateset for Logistic Regression: 0.9188214285714286
AUC Score on training dateset for Logistic Regression: 0.9737487548005233
F1 Score ftraining dateset for Logistic Regression: 0.9188199999589727
Precision Score on test for Logistic Regression: 0.896
AUC Score on test for Logistic Regression: 0.9606303677080659
F1 Score for Logistic Regression: 0.8960019821539781


In [ ]:
model_2 = Pipeline(
    steps=[
        # ("classifier", DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)),
        ("classifier", DecisionTreeClassifier())
    ]
)


In [ ]:
model_2.fit(x_train_tfidf,y_train)

In [ ]:
print("Precision Score on training dateset for Decision Tree Classifier: %s" %
      precision_score(y_train, model_2.predict(x_train_tfidf), average='micro'))
print("AUC Score on training dateset for Decision Tree Classifier: %s" % roc_auc_score(
    y_train, model_2.predict_proba(x_train_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_train_2 = f1_score(y_train, model_2.predict(
    x_train_tfidf), average="weighted")
print("F1 Score training dateset for Decision Tree Classifier: %s" %
      f1_score_train_2)
print("Precision Score on test for Decision Tree Classifier: %s" %
      precision_score(y_test, model_2.predict(x_test_tfidf), average='micro'))
print("AUC Score on test for Decision Tree Classifier: %s" % roc_auc_score(
    y_test, model_2.predict_proba(x_test_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_2 = f1_score(y_test, model_2.predict(
    x_test_tfidf), average="weighted")
print("F1 Score for Decision Tree Classifier: %s" % f1_score_2)


In [ ]:
model_3 = Pipeline(
    steps=[
        ("classifier", DecisionTreeClassifier(criterion='gini',
         max_depth=11, min_samples_split=2, min_samples_leaf=1)),
    ]
)


In [ ]:
model_3.fit(x_train_tfidf,y_train)

In [ ]:
print("Precision Score on training dateset for Decision Tree Classifier: %s" %
      precision_score(y_train, model_3.predict(x_train_tfidf), average='micro'))
print("AUC Score on training dateset for Decision Tree Classifier: %s" % roc_auc_score(
    y_train, model_3.predict_proba(x_train_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_train_3 = f1_score(y_train, model_3.predict(
    x_train_tfidf), average="weighted")
print("F1 Score training dateset for Decision Tree Classifier: %s" %
      f1_score_train_3)
print("Precision Score on test for Decision Tree Classifier: %s" %
      precision_score(y_test, model_3.predict(x_test_tfidf), average='micro'))
print("AUC Score on test for Decision Tree Classifier: %s" % roc_auc_score(
    y_test, model_3.predict_proba(x_test_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_3 = f1_score(y_test, model_3.predict(
    x_test_tfidf), average="weighted")
print("F1 Score for Decision Tree Classifier: %s" % f1_score_3)


In [ ]:
model_4 = Pipeline(
    steps=[
        # ("classifier", RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=15, min_samples_split=3, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None))
        ("classifier", RandomForestClassifier())]
)


In [ ]:
model_4.fit(x_train_tfidf,y_train)

In [ ]:
print("Precision Score on training dateset for Random Forest Classifier: %s" %
      precision_score(y_train, model_4.predict(x_train_tfidf), average='micro'))
print("AUC Score on training dateset for Random Forest Classifier: %s" % roc_auc_score(
    y_train, model_4.predict_proba(x_train_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_train_4 = f1_score(y_train, model_4.predict(
    x_train_tfidf), average="weighted")
print("F1 Score training dateset for Random Forest Classifier: %s" %
      f1_score_train_4)
print("Precision Score on test for Random Forest Classifier: %s" %
      precision_score(y_test, model_4.predict(x_test_tfidf), average='micro'))
print("AUC Score on test for Random Forest Classifier: %s" % roc_auc_score(
    y_test, model_4.predict_proba(x_test_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_4 = f1_score(y_test, model_4.predict(
    x_test_tfidf), average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_4)


In [ ]:
model_5 = Pipeline(
    steps=[
        ("classifier", AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),
                                          n_estimators=100,
                                          learning_rate=.8)),
    ]
)


In [ ]:
model_5.fit(x_train_tfidf,y_train)

In [ ]:
print("Precision Score on training dateset for Ada Boost Classifier: %s" %
      precision_score(y_train, model_5.predict(x_train_tfidf), average='micro'))
print("AUC Score on training dateset for Ada Boost Classifier: %s" % roc_auc_score(
    y_train, model_5.predict_proba(x_train_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_train_5 = f1_score(y_train, model_5.predict(
    x_train_tfidf), average="weighted")
print("F1 Score training dateset for Ada Boost Classifier: %s" % f1_score_train_5)
print("Precision Score on test for Ada Boost Classifier: %s" %
      precision_score(y_test, model_5.predict(x_test_tfidf), average='micro'))
print("AUC Score on test for Ada Boost Classifier: %s" % roc_auc_score(y_test,
      model_5.predict_proba(x_test_tfidf)[:, 1], multi_class='ovo', average='macro'))
f1_score_5 = f1_score(y_test, model_5.predict(
    x_test_tfidf), average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_5)
